IMPORTS 

In [2]:
import pickle
import json
import pandas as pd

LOAD THE MODEL

In [3]:
with open("models/best_model.pkl", 'rb') as f:
    model = pickle.load(f)
    
with open("models/model_metadata.json", 'r') as f:
    metadata = json.load(f)

PREDICTION FUNCTION

GET THE FEATURES FROM META DATA

In [8]:
expected_features = metadata['features']

In [10]:
print("Model expects these features:", metadata['features'])

Model expects these features: ['Date', 'Close', 'High', 'Low', 'Open', 'Volume', 'avg_sentiment', 'tweet_count', 'next_day_change', '5_day_ma', '10_day_volatility', 'daily_return', 'sentiment_lag1', 'sentiment_ma3']


In [ ]:
def predict(features):
    """
    Predicts stock movement with feature validation
    Args:
        features (dict): Dictionary containing ALL required features
    Returns:
        dict: Prediction results
    """
    # Get model's expected features (excluding metadata columns)
    expected_features = [f for f in metadata['features'] 
                       if f not in ['Date', 'next_day_change']]
    
    # Validate input features
    missing_features = set(expected_features) - set(features.keys())
    if missing_features:
        raise ValueError(f"Missing required features: {missing_features}")
    
    # Create DataFrame in correct feature order
    input_df = pd.DataFrame({f: [features[f]] for f in expected_features})
    
    # prediction
    pred = model.predict(input_df)[0]
    
    return {
        'predicted_change': float(pred),
        'direction': 'UP' if pred > 0 else 'DOWN',
        'confidence': min(99, abs(pred)*100)
    }

In [13]:
# Create complete test input
test_features = {
    'High': 255.50,          # Example value
    'Low': 248.20,           # Example value
    'Open': 252.30,          # Example value
    'Close': 250.00,         # Example value
    'Volume': 5000000,       # Example value
    'daily_return': 0.015,   # Example value
    'avg_sentiment': 0.15,
    'tweet_count': 85,
    '5_day_ma': 250.50,
    '10_day_volatility': 0.02,
    'sentiment_lag1': 0.12,
    'sentiment_ma3': 0.14
}


test_features = {k: test_features[k] for k in expected_features}

# Make prediction
try:
    prediction = predict(test_features)
    print("📈 Prediction Result:")
    print(prediction)
except ValueError as e:
    print(f"❌ Error: {str(e)}")

📈 Prediction Result:
{'predicted_change': 0.006560678593814373, 'direction': 'UP', 'confidence': 0.6560678593814373}
